In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0

# 设置参数
num_classes = 10  # CIFAR-10有10个类别
input_shape = (32, 32, 3)  # CIFAR-10图像的输入形状

# 加载CIFAR-10数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 数据预处理
mean = x_train.mean(axis=(0, 1, 2), keepdims=True)
std = x_train.std(axis=(0, 1, 2), keepdims=True)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# 将标签转换为分类格式
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 调整输入尺寸以匹配EfficientNet的预期输入（224x224）
x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))

# 构建EfficientNet模型
def build_efficientnet(input_shape, num_classes):
    # 加载预训练的EfficientNetB0模型，不包括顶层
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

    # 冻结预训练模型的权重
    base_model.trainable = False

    inputs = keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

# 构建EfficientNet模型
efficientnet_model = build_efficientnet((224, 224, 3), num_classes)

# 编译模型
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 打印模型摘要
efficientnet_model.summary()

# 训练模型（可以先进行微调，再解冻部分层进行训练）
efficientnet_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# 解冻部分层，进行微调
base_model = efficientnet_model.layers[1]
base_model.trainable = True

# 重新编译模型（当改变可训练层时需要重新编译）
efficientnet_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# 继续训练模型
efficientnet_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# 评估模型
loss, accuracy = efficientnet_model.evaluate(x_test, y_test)
print(f"测试损失: {loss:.4f}, 测试准确率: {accuracy:.4f}")
